In [0]:
# default_exp data.exploratory

# Exploration of your data

> This module comprises all the statistical and inference techniques to describe the inner properties of software data. The submodules might include:
>
> - Descriptive statistics
> - Software Metrics
> - Information Theory
> - Learning Principels Detection (Occams' Razor, Biased data, and Data Snooping)
> - Inference: Probabilistic and Causal

In [0]:
# export
# Imports
import pandas as pd
# import sentencepiece as sp

In [0]:
#hide
from nbdev.showdoc import *

In [0]:
#Import dataframe from MongoDB
def get_dataframes_from_mongo():
  requirements = {'file_name': [], 'contents': []}
  path = "./requirements"
  for file in os.listdir(path):
    requirements['file_name'].append(file)
    with open (os.path.join(path, file), "r") as f:
      requirements['contents'].append(f.read())
  source_code = {'file_name': [], 'contents': []}
  path = "./source_code"
  for file in os.listdir("./source_code"):
    source_code['file_name'].append(file)
    with open (os.path.join(path, file), "r") as f:
      source_code['contents'].append(f.read())
  req_df = pd.DataFrame(data = requirements)
  src_df = pd.DataFrame(data = source_code)

  return req_df, src_df

a, b = get_dataframes_from_mongo()

/content
['RQ11.txt', 'RQ24.txt', 'RQ38.txt', 'RQ4.txt', 'RQ18.txt', 'RQ22.txt', 'RQ21.txt', 'RQ26.txt', 'RQ40.txt', 'RQ58.txt', 'RQ37.txt', 'RQ39.txt', 'RQ14.txt', 'RQ7.txt', 'RQ31.txt', 'RQ56.txt', 'RQ48.txt', 'RQ9.txt', 'RQ45.txt', 'RQ36.txt', 'RQ33.txt', 'RQ57.txt', 'RQ51.txt', 'RQ16.txt', 'RQ5.txt', 'RQ29.txt', 'RQ19.txt', 'RQ34.txt', 'RQ25.txt', 'RQ32.txt', 'RQ46.txt', 'RQ13.txt', 'RQ53.txt', 'RQ15.txt', 'RQ20.txt', 'RQ50.txt', 'RQ2.txt', 'RQ27.txt', 'RQ8.txt', 'RQ55.txt', 'RQ35.txt', 'RQ23.txt', 'RQ17.txt', 'RQ49.txt', 'RQ42.txt', 'RQ28.txt', 'RQ41.txt', 'RQ10.txt', 'RQ1.txt', 'RQ6.txt', 'RQ52.txt', 'RQ47.txt']


In [0]:
#Compute entropy of all the files per system and calculate mean, std, median, and std for median absolute deviation. The idea is to create confidence intervals for each system/dataset


In [0]:
#Rank the system/datasets according to the confidence intervals
#Compute the confidence intervals for all cross-entropy values
#Rank the systems/datasets according to cross-entropy values
#Top 50 most frequent tokens of each system and corpus (one system has generally two corpora)
#Top 50 least frequent tokes of each system and corpus
#What are the tokens that are in the target and not in the source (and the other way around)? Compute the distribution for those tokens
#What are the mutual tokens (source and target)? please compute distribution
#-Compute confidence intervals for the software metrics on source code (e.g., cyclo, loc, lcom5)


In [0]:
Run metrics

In [0]:
Visualize

In [0]:
Push updated fields to Mongo